In [14]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

INPUT_NODE=784#输入层的节点数等于输入图片的像素，mnist是28X28=784
OUTPUT_NODE=10

LAYER1_node=500
BATCH_SIZE=100

LEARNING_RATE_BASE=0.8
LEARNING_RATE_DECAY=0.99

REGULARIZATION_RATE=0.0001
TRAINING_STEPS=30000
MOVING_AVERAGE_DECAY=0.99

def inference(input_tensor,avg_class,weights1,biases1,weights2,biases2):
    if avg_class==None:
        layer1=tf.nn.relu(tf.matmul(input_tensor,weights1)+biases1)
        return tf.matmul(layer1,weight2)+biases2
    else:
        layer1=tf.nn.relu(tf.matmul(input_tensor,avg_class.average(weights1))+avg_class.average(biases1))
        return tf.matmul(layer1,avg_class.average(weights2))+avg_class.average(biases2)
    
#训练模型的过程
def train(mnist):
    x =tf.placeholder(tf.float32,[None,INPUT_NODE],name='x-input')
    y_=tf.placeholder(tf.float32,[None,OUTPUT_NODE],name='y-input')
    #生成隐含层的参数
    weights1=tf.Variable(tf.truncated_normal([INPUT_NODE,LAYER1_NODE],sttddev=0.1))
    biases1=tf.Varialble(tf.constant(0.1,shape=[LAYER1_NODE]))
    #生层输出层的参数
    weights2=tf.Variable(tf.truncated_normal([LAYER1_NODE,OUTPUT_NODE],stddev=0.1))
    biases2=tf.Variable(tf.constant(0.1,shape=[OUTPUT_NODE]))
    #计算当前前向计算的结果，这里参数为NONE，所以不用滑动平均
    y=inference(x,None,weights1,biases1,weights2,biases2)
    
    #训练轮数不参与训练
    global_step=tf.Variable(0,trainable=False)
    
    #给定滑动平均衰减率和训练轮数的变量，初始化滑动平均类
    variable_averages=tf.train.ExponentialMovingAveerage(MOVING_AVERAGE_DECAY,global_step)
    #在所有代表伸进网络参数的变量上使用滑动平均
    variables_averages_op=variable_averages.apply(tf.trainable_variables())
    #计算使用滑动平均之后的前向传播结果
    average_y=inference(x,variable_averages,weights1,biases1,weights2,biases2)
    
    #用交叉煽来刻画预测值和真实值之间的差距的损失函数，用rensorflow提供的sparse_softmax_cross_with_logits计算交叉煽
    cross_entropy=tf.nn.sparse_softmax_cross_with_logits(logits=y,labels=tf.argmax(y_, 1))
    cross_entropy_mean=tf.reduce_mean(cross_entropy)
    
    #计算L2正则化损失函数
    regularizer=tf.contrib.layers.l2_regularizer(REGULARIZATION_RATE)
    #计算模型的正则化损失，一般只计算权重，不计算偏置
    regularization=regularizer(weights1)+regularizer(weights2)
    #总损失等于交叉煽损失和正则化损失之和
    loss=regularization+cross_entropy_mean
    #设置指数衰减的学习率
    learning_rate=tf.train.exponential_decay(LEARNING_RATE_BASE,global_step,mnist.train.num_examples/BATCH_SIZE,LEARNING_RATE_DECAY)
    
    #用梯度下降来优化损失函数
    train_step=tf.train.GradientDescentOptimizer(learning_rate)\
    .minimize(loss,global_step=global_step)
    #在训练神经网络模型的时候，每过一遍数据既需要通过反响传播来更新神经网络中的参数，又要更新每一个参数的滑动平均值，为了一次完成两个操作
    #提供tf.group或者control_dependencies
    train_op=tf.group(train_step,variables_averages_op)
    #为了检验滑动平均模型的神经网络前向传播
    correct_prediction=tf.equal(tf.argmax(average_y, 1),tf.argmax(y_,1))
    accuracy=tf.reduce_mean(tf.cast(correct_prediction,tf,float32))#将布尔值转换为实数值

#初始化会话开始训练
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    validate_feed={x :mnist.validation.images,y_:mnist.validation.labels}
    tset_feed={x :mnist.test.images,y_:mnist.test.labels}
    
    for i in range(TRAINING_STEPS):
        if i%1000==0:
            validate_acc=sess.run(accuracy,feed_dict=validate_feed)
            print("After %d training steps,validation accuracy" "using average model is %g"%(i,validate_acc))
            
        xs,ys=mnisttrain.next_batch(BATCH_SIZE)
        sess.run(train_op,feed_dict={x: xs,Y_: ys})
    
    #训练结束后
    test_acc=sess.run(accuracy,feed_dict=test_feed)
    print("After %d training steps,validation accuracy" "using average model is %g"%(TRAINING_STEPS,test_acc))
    
def main(argv=None):
    mnist=input_data.read_data_sets("/tmp/data",one_hot=True)
    train(mnist)
if __name__=='__main__':
    tf.app.run()

    
    
    

NameError: name 'x' is not defined